# MNIST Experiments - Playing With Different Architectures

Sometimes we need to come back to the basis, this is the place I choose for that.

Here I'll experiment with different networks on the MNIST and MNIST variants datasets trying to find relations in which I can reduce the number of parameters in comparison with a Fully Connected (FC) network.

Later on, I might try with other datasets that are small enough for my GTX1080.

Yes, I know, the issue is already solved for Images with Convolutional Networks but what I want to see is not that. Instead I want to understand ways in which fully connected networks can be replaced by other types of connections to minimize the number of parameters in it. This is an exploratory work to get a deeper understanding on Neural Networks (NNs) that will at least give me some fun time.



## SparseNet

MMMMmmmmmm ... something is not working right here .... should fix it ...
where have I made the mistake ... ?


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
# Network modules  to try
from network_modules import *
from net_utils import *

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def transform_mnist():
    return transforms.Compose([
#         transforms.Grayscale(),
#         transforms.Resize((w, h)),  # this should be used ONLY if the image is bigger than this size
        transforms.ToTensor()
#         transforms.Normalize(0.5, 0.25)
    ])

In [6]:

def train(model, mname, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):

    model.to(device)
    num_epochs = 100
    batch_size = 128
#     learning_rate = 0.0001
    learning_rate = 0.001
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

#     criterion = nn.MSELoss()
    criterion = F.nll_loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

    transformation = transform_mnist()
    train_loader, test_loader = get_loaders(batch_size, transformation)


    for epoch in range(num_epochs):
        for i, (img, labels) in enumerate(train_loader):
#             print("shape: ", img.shape, labels.shape)
            labels = labels.to(device)
            img = img.to(device).view((-1,784))
            
#             print("shape2: ", img.shape)
            # ===================forward=====================
            #         print("encoding batch of  images")
            output = model(img)
#             print("output shape: ", output.shape, labels.shape, labels[:10])
            #         print("computing loss")
            loss = criterion(output, labels)
            # ===================backward====================
            #         print("Backward ")
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # ===================log========================
        if epoch % 20 == 0:
            print('epoch [{}/{}], loss:{:.6f}'.format(epoch+1, num_epochs, loss.data[0]))
#         if epoch % 10 == 0:
#             pic = to_img(output.cpu().data)
#             in_pic = to_img(img.cpu().data)
#             save_image(pic, './results/2x2-out_image_{}.png'.format(epoch))
#             save_image(in_pic, './results/2x2-in_image_{}.png'.format(epoch))
#         if loss.data[0] < 0.015: #arbitrary number because I saw that it works well enough
#             print("loss < 0.015, breaking")
#             break
#     model.save_model(mname, "model")

    print('########################################################')
    print('Final performance of model {} epoch [{}/{}], loss:{:.8f}'.format(mname, epoch+1, num_epochs, loss.data[0]))
    print('--------------------------------------------------------')


In [7]:
activations = [
    "relu",
    "relu6",
    "sigmoid",
    "elu",
    "leaky_relu",
    "logsigmoid",
    ]

In [12]:
fcnets_layers = [
        [784,500,10],
        [784,1000,10],
        [784,1500,10],
        [784,500,500,10],
        [784,1000,500,10],
        [784,1000,1000,10],
        [784,500,500,500,10],
        [784,1000,500,500,10],
        [784,1000,1000,500,10],
        [784,1000,1000,1000,10],
]

sparsities = [0.9, 0.8, 0.7, 0.6]  # , 0.5, 0.4, 0.3, 0.2, 0.1]  # [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

In [13]:

models = []
for l in fcnets_layers:
    for s in sparsities:
#         for a in activations:
        for a in ["relu"]:
            mname = str(l)+"_"+str(s)+"_"+a
            models.append(SparseNet(l, sparsity=s, activation=a, name=mname) )

In [14]:
len(models)

40

In [15]:
for i in range(len(fcnets_layers)):
    model = models[i]
    mname = model.name
    train(model, mname)
    

epoch [1/100], loss:-22719.091797


/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [21/100], loss:-6152859.500000
epoch [41/100], loss:-20490514.000000
epoch [61/100], loss:-47534016.000000
epoch [81/100], loss:-77134024.000000
########################################################
Final performance of model [784, 500, 10]_0.9_relu epoch [100/100], loss:-115477160.00000000
--------------------------------------------------------


/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/ipykernel_launcher.py:52: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [1/100], loss:-19076.304688
epoch [21/100], loss:-5708181.500000
epoch [41/100], loss:-19475594.000000


Process Process-504:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/torchvision/datasets/mnist.py", line 77, in __getitem__
    img = self.transform(img)
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/torchvision/transforms/transforms.py", line 49, in __call__
    img = t(img)
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/tor

Traceback (most recent call last):
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-90bff572eccb>", line 4, in <module>
    train(model, mname)
  File "<ipython-input-6-555887a5db65>", line 21, in train
    for i, (img, labels) in enumerate(train_loader):
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 330, in __next__
    idx, batch = self._get_batch()
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 309, in _get_batch
    return self.data_queue.get()
  File "/usr/lib/python3.5/queue.py", line 164, in get
    self.not_empty.wait()
  File "/usr/lib/python3.5/threading.py", line 293, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call la

KeyboardInterrupt: 

In [11]:
for i in range(len(fcnets_layers)):
    model = models[i]
    mname = model.name
    train(model, mname)
    

epoch [1/100], loss:-3690.978271


/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [21/100], loss:-933181.437500
epoch [41/100], loss:-3140060.250000
epoch [61/100], loss:-6421920.500000
epoch [81/100], loss:-11181103.000000
########################################################
Final performance of model [784, 500, 10]_0.1_relu epoch [100/100], loss:-16312471.00000000
--------------------------------------------------------


/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/ipykernel_launcher.py:52: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [1/100], loss:-5665.460938
epoch [21/100], loss:-1565372.000000
epoch [41/100], loss:-5051212.000000
epoch [61/100], loss:-10459762.000000
epoch [81/100], loss:-18005732.000000
########################################################
Final performance of model [784, 500, 10]_0.2_relu epoch [100/100], loss:-28013096.00000000
--------------------------------------------------------
epoch [1/100], loss:-7268.011719
epoch [21/100], loss:-2069410.125000
epoch [41/100], loss:-6789892.500000
epoch [61/100], loss:-14524044.000000
epoch [81/100], loss:-24793238.000000
########################################################
Final performance of model [784, 500, 10]_0.3_relu epoch [100/100], loss:-36976052.00000000
--------------------------------------------------------
epoch [1/100], loss:-10079.895508
epoch [21/100], loss:-3124085.750000
epoch [41/100], loss:-10146665.000000


Process Process-354:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/torchvision/datasets/mnist.py", line 77, in __getitem__
    img = self.transform(img)
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/torchvision/transforms/transforms.py", line 49, in __call__
    img = t(img)
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/tor

Traceback (most recent call last):
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-90bff572eccb>", line 4, in <module>
    train(model, mname)
  File "<ipython-input-6-555887a5db65>", line 21, in train
    for i, (img, labels) in enumerate(train_loader):
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 336, in __next__
    return self._process_next_batch(batch)
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 355, in _process_next_batch
    self._put_indices()
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 345, in _put_indices
    indices = next(self.sample_iter, None)
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/torch/utils/data/sampler.py", line 139, 

KeyboardInterrupt: 